In [1]:
import pathlib
import time
import tempfile
import os

import census
import us
import cpi
import datetime
import pandas as pd
# cpi.update()  # on first run

import plotly.express as px
import plotly.subplots
import plotly.graph_objects as go

import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from tqdm.auto import tqdm

api_key = pathlib.Path('api_key_secret.txt').read_text().strip()
print('api_key: "' + api_key + '"')

c = census.Census(api_key)

api_key: "724a9e800f0a47d3648a2d76bc32afb87e0f408c"


In [2]:
# https://data.census.gov/cedsci/table?q=gross%20rent%20san%20francisco&g=0100000US&tid=ACSDP1Y2021.DP04

start_year = 2010  # Earliest year data available
end_year = datetime.date.today().year
stats_of_interest = {
    'B25064_001E': 'Median Rent',
    'B25077_001E': 'Median Value',
}

In [21]:
census_data_csv_cache_path = os.path.join(tempfile.gettempdir(), 'census_data.csv')
if os.path.exists(census_data_csv_cache_path):
    census_data = pd.read_csv(census_data_csv_cache_path)
else:
    search_years = list(range(start_year, end_year+1))
    census_data = []
    reporter = tqdm(total=len(stats_of_interest)*len(search_years))
    for stat_id, stat_name in stats_of_interest.items():
        for year in search_years:
            try:
                df = pd.DataFrame.from_records(c.acs1.get(('NAME', stat_id), {'for': 'county:*', 'in': 'state:*'}, year=year))
            except Exception as e:
                print("Caught exception for stat_name, year, stat_id:", stat_name, year, stat_id)
                print(e)
                time.sleep(0.1)
                reporter.update()
                continue
            df = df.rename(columns={stat_id: 'Value', 'NAME': 'Name', 'state': 'State', 'county': 'County'})
            df['Statistic'] = stat_name
            df['Stat ID'] = stat_id
            df['Year'] = year
            census_data.append(df)
            time.sleep(0.1)
            reporter.update()
    reporter.close()
    census_data = pd.concat(census_data, ignore_index=True)
    census_data.to_csv(census_data_csv_cache_path, index=False)

# Merge state names into census_data
state_info_df = []
for state in us.states.STATES:
    state_info_df.append({'State': state.fips, 'State Name': state.name, 'State Abbreviation': state.abbr})
state_info_df = pd.DataFrame(state_info_df)
state_info_df['State'] = state_info_df['State'].astype(int)
census_data['State'] = census_data['State'].astype(int)
census_data = census_data.merge(state_info_df, on='State')

print(len(census_data))
census_data.head()

21368


,Name,Value,State,County,Statistic,Stat ID,Year,State Name,State Abbreviation
0,"Baldwin County, Alabama",832.0,1,3,Median Rent,B25064_001E,2010,Alabama,AL
1,"Calhoun County, Alabama",607.0,1,15,Median Rent,B25064_001E,2010,Alabama,AL
2,"Cullman County, Alabama",585.0,1,43,Median Rent,B25064_001E,2010,Alabama,AL
3,"DeKalb County, Alabama",502.0,1,49,Median Rent,B25064_001E,2010,Alabama,AL
4,"Elmore County, Alabama",847.0,1,51,Median Rent,B25064_001E,2010,Alabama,AL


In [46]:
merge_cols = ['Name', 'State', 'County', 'Year', 'State Name', 'State Abbreviation']
census_wide = census_data[merge_cols].drop_duplicates()
for stat_name in census_data['Statistic'].drop_duplicates():
    census_wide = census_wide.merge(census_data.loc[census_data['Statistic'] == stat_name].drop(columns=['Statistic', 'Stat ID']).rename(columns={'Value': stat_name}), on=merge_cols, how='left')
for stat_name in census_data['Statistic'].drop_duplicates():
    census_wide[stat_name+' ({} dollars)'.format(cpi.LATEST_YEAR)] = None
    for year in census_wide['Year'].drop_duplicates():
        # cpi is slow so we calculate the inflation factor once and then apply it to all rows for that year
        inflation_factor = cpi.inflate(1, year, to=cpi.LATEST_YEAR)
        census_wide.loc[census_wide['Year'] == year, stat_name+' ({} dollars)'.format(cpi.LATEST_YEAR)] = census_wide.loc[census_wide['Year'] == year, stat_name] * inflation_factor
    assert not census_wide[stat_name+' ({} dollars)'.format(cpi.LATEST_YEAR)].isnull().any()
census_wide['Price-to-Rent Ratio ({} dollars)'.format(cpi.LATEST_YEAR)] = census_wide['Median Value ({} dollars)'.format(cpi.LATEST_YEAR)] / (census_wide['Median Rent ({} dollars)'.format(cpi.LATEST_YEAR)] * 12.0)
print(len(census_wide))

# Add median per state
census_wide_state = census_wide.drop(columns=['Name', 'County']).groupby(['State Name', 'State Abbreviation', 'State', 'Year']).median().reset_index()
census_wide_state['Name'] = census_wide_state['State Name'] + ' Median'
display(census_wide_state.loc[census_wide_state['Year'] == census_wide_state['Year'].max()].sort_values('Median Rent').head())
# Now median all of us by year
census_wide_us = census_wide_state.drop(columns=['Name', 'State Name', 'State Abbreviation', 'State']).groupby(['Year']).median().reset_index()
census_wide_us['Name'] = 'US Median'
display(census_wide_us.tail())
census_wide = pd.concat([census_wide, census_wide_state, census_wide_us], ignore_index=True)

census_wide.loc[census_wide['Name'] == 'San Francisco County, California'].sort_values('Year')

10684


,State Name,State Abbreviation,State,Year,Median Rent,Median Value,Median Rent (2023 dollars),Median Value (2023 dollars),Price-to-Rent Ratio (2023 dollars),Name
623,West Virginia,WV,54,2023,859.0,174900.0,859.0,174900.0,17.688107,West Virginia Median
454,Ohio,OH,39,2023,932.0,209500.0,932.0,209500.0,18.342949,Ohio Median
441,North Dakota,ND,38,2023,958.5,275550.0,958.5,275550.0,23.943415,North Dakota Median
12,Alabama,AL,1,2023,966.0,216850.0,966.0,216850.0,20.251163,Alabama Median
51,Arkansas,AR,5,2023,968.0,220900.0,968.0,220900.0,18.707265,Arkansas Median


,Year,Median Rent,Median Value,Median Rent (2023 dollars),Median Value (2023 dollars),Price-to-Rent Ratio (2023 dollars),Name
8,2018,874.5,189025.0,1061.148829,229369.533904,18.284219,US Median
9,2019,891.5,198200.0,1062.524527,236222.502806,18.432122,US Median
10,2021,974.0,240350.0,1095.249467,270270.23545,20.035557,US Median
11,2022,1106.5,288500.0,1152.048532,300375.961456,20.736516,US Median
12,2023,1174.0,307300.0,1174.0,307300.0,20.992572,US Median


,Name,State,County,Year,State Name,State Abbreviation,Median Rent,Median Value,Median Rent (2023 dollars),Median Value (2023 dollars),Price-to-Rent Ratio (2023 dollars)
606,"San Francisco County, California",6.0,75.0,2010,California,CA,1385.0,768000.0,1935.33895,1073169.90131,46.209386
646,"San Francisco County, California",6.0,75.0,2011,California,CA,1407.0,719800.0,1905.919889,975039.897928,42.632078
686,"San Francisco County, California",6.0,75.0,2012,California,CA,1512.0,727600.0,2006.626584,965622.687004,40.101411
726,"San Francisco County, California",6.0,75.0,2013,California,CA,1491.0,778000.0,1950.19116,1017604.776847,43.483121
766,"San Francisco County, California",6.0,75.0,2014,California,CA,1587.0,846800.0,2042.621629,1089913.040687,44.465448
806,"San Francisco County, California",6.0,75.0,2015,California,CA,1659.0,941400.0,2132.761017,1210235.817684,47.287523
846,"San Francisco County, California",6.0,75.0,2016,California,CA,1784.0,1024000.0,2264.885474,1300023.949301,47.832586
886,"San Francisco County, California",6.0,75.0,2017,California,CA,1836.0,1104100.0,2282.281625,1372476.657148,50.113471
926,"San Francisco County, California",6.0,75.0,2018,California,CA,1880.0,1195700.0,2281.257631,1450904.122147,53.000887
966,"San Francisco County, California",6.0,75.0,2019,California,CA,1959.0,1217500.0,2334.81273,1451064.062396,51.79088


In [47]:
plot_counties = [
    'San Francisco County, California',
    'Santa Clara County, California',
    'San Diego County, California',
    # 'Los Angeles County, California',
    # 'Hennepin County, Minnesota',
    # 'Bay County, Florida',
    'Miami-Dade County, Florida',
    'Wayne County, Michigan',
    'US Median',
    'California Median',
    'New Hampshire Median',
]

fig = px.line(census_wide.loc[census_wide['Name'].isin(plot_counties)], x='Year', y='Price-to-Rent Ratio ({} dollars)'.format(cpi.LATEST_YEAR), color='Name', title='Price-to-Rent Ratio ({} dollars)'.format(cpi.LATEST_YEAR))
fig.update_layout(width=1000, height=500)
fig.show()

In [48]:
for plot_county in plot_counties:
    df = census_wide.loc[census_wide['Name'] == plot_county].copy()

    # Convert above plot to Plotly showing both lines on independent y axes
    fig = plotly.subplots.make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Scatter(x=df['Year'], y=df['Median Rent ({} dollars)'.format(cpi.LATEST_YEAR)], mode='lines', name='Rent', line=dict(color='blue')), secondary_y=False)
    fig.add_trace(go.Scatter(x=df['Year'], y=df['Median Value ({} dollars)'.format(cpi.LATEST_YEAR)], mode='lines', name='Value', line=dict(color='red')), secondary_y=True)
    fig.update_layout(width=1000, height=500)
    fig.update_xaxes(title_text='Year')
    fig.update_yaxes(title_text='Median Rent ({} dollars)'.format(cpi.LATEST_YEAR), secondary_y=False)
    fig.update_yaxes(title_text='Median Value ({} dollars)'.format(cpi.LATEST_YEAR), secondary_y=True)
    fig.update_layout(title=plot_county)
    fig.show()
